In [ ]:
import os
import sys
import json

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
d = glhe.globals.functions.load_json('MFRTRT_STS_input.json')
d['ground-heat-exchanger'][0]['g-function-path'] = norm(join(cwd, '..', 'MFRTRT_EWT_g_functions', 'EWT_experimental_g_functions.csv'))
d['ground-heat-exchanger'][0]['load-aggregation']['method'] = 'dynamic'
d['simulation']['time-steps-per-hour'] = 60
d['simulation']['runtime'] = 3600 * 1

In [ ]:
with open(join(cwd, 'in.json'), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in.json').simulate()

In [ ]:
df = pd.read_csv('out.csv', index_col=0, parse_dates=True)

In [ ]:
styles = ['bs-','ro-','y^--', 'gx--']

df[['ConstantLoad:CONSTANT 4000:Inlet Temp. [C]', 
    'ConstantLoad:CONSTANT 4000:Outlet Temp [C]',
    'GroundHeatExchangerSTS:GHE 1:Inlet Temp. [C]',
    'GroundHeatExchangerSTS:GHE 1:Outlet Temp [C]']].plot(style=styles)

In [ ]:
def plot_it(timestamp):
    t_in = []
    t_out = []
    depth = 76.2
    num_segs = 10
    dz = depth / num_segs
    d = np.arange(dz , depth + dz, dz)
    for i in range(10):
        val_in = df['SegmentUTubeBHGrouted:Seg No. {}:Inlet Temp Leg 1 [C]'.format(i+1)].loc[timestamp]
        val_out = df['SegmentUTubeBHGrouted:Seg No. {}:Outlet Temp Leg 1 [C]'.format(i+1)].loc[timestamp]
        t_in.append(val_in)
        t_out.append(val_out)

    plt.plot(t_in, d, label='Leg 1')
    plt.plot(t_out, d, label='Leg 2', linestyle='--')
    plt.ylim(80, 0)
    plt.ylabel('Depth [m]')
    plt.xlabel('Temperature [C]')
    plt.legend()

In [ ]:
plot_it('2019-01-01 00:10:00')

In [ ]:
Tg = 16.1
df['g'] = (df['GroundHeatExchangerSTS:GHE 1:Outlet Temp [C]'] - Tg) * 2 * 3.14159 * 2.7 / (4000 / 76.2)

In [ ]:
df['g'].plot()